In [1]:
from pdf2image import convert_from_path
import csv
import glob
import os
import shutil

In [5]:
# File and Directory Paths
dataset_file = "extracted_tables/tables.csv"
source_dir = "source_files/"
source_metadata_file = source_dir + "papers.csv"
output_dir = "extracted_tables/table_images/"
  
os.makedirs(output_dir, exist_ok=True)

In [31]:
"""
Converting latex code tables to png tables
Start - index of the first paper to be used (beginning from 0)
End - index of the last paper to be used
"""
def extract_data_from_papers(start, end):
    csvfile_metadata = open(source_metadata_file, "r")
    spamreader = csv.reader(csvfile_metadata, delimiter=';', quotechar='"', quoting=csv.QUOTE_ALL)
    
    for index, row in enumerate(spamreader):
        if index < start:
            continue
        if index > end:
            break
            
        paper_id = row[0]
        paper_path = source_dir + paper_id
        print(paper_path)
        if os.path.isdir(paper_path):
            table_files = [x for x in os.listdir(paper_path) if x.startswith('FR_TAB_')]
            print(str(index) + ", " + paper_id + ": " + str(len(table_files)) + " tables")

            for table in table_files:
                try:
                    # Using pdflatex to generate pdf file
                    os.system(f"pdflatex -interaction=nonstopmode -output-directory={paper_path} {os.path.join(paper_path, table)}")

                    # Converting pdf file to an image
                    table = table.replace(".tex", ".pdf")
                    images = convert_from_path(os.path.join(paper_path, table))              
                    table = table.replace(".pdf", ".png")            
                    saved_png_path = os.path.join(paper_path, table)
                    table_id = table.replace("FR_TAB_", "")
                    images[0].save(saved_png_path, "PNG")

                    # Trimming whitespace
                    os.system(f"magick {saved_png_path} -trim {output_dir + table_id}")

                except Exception as e:
                    print(f"Error Type: {type(e).__name__}")
                    error_message = str(e)[:100]
                    print(f"Error Message: {error_message}")

            table_files = [x for x in os.listdir(paper_path) if x.startswith('FR_TAB_') and x.endswith(".tex") == False]
            for t_file in table_files:
                os.remove(os.path.join(paper_path, t_file))

In [23]:
extract_data_from_papers(10, 631)

source_files/2312.15124
10, 2312.15124: 1 tables
source_files/2401.03492
11, 2401.03492: 0 tables
source_files/2401.03820
12, 2401.03820: 0 tables
source_files/2401.13246
13, 2401.13246: 5 tables


KeyboardInterrupt: 

In [33]:
extract_data_from_papers(106, 106)

source_files/2409.18268
106, 2409.18268: 1 tables
Error Type: PDFPageCountError
Error Message: Unable to get page count.
I/O Error: Couldn't open file 'source_files/2409.18268\FR_TAB_2409.18268_T
